In [1]:
#
# Import Libraries
#

import numpy as np
import os
import pandas as pd
import sys
from dotmap import DotMap

from plotly.subplots import make_subplots
import plotly.graph_objects as go
from plotly.colors import n_colors

utilsPath = r'S:\U_Proteomica\UNIDAD\software\MacrosRafa\data\Metabolomics\PESA_Integromics\Data\utils'
if utilsPath not in sys.path:
    sys.path.append(utilsPath)

In [2]:
#
# Constants
#

modes = ['cp', 'cn']

xm_path = r"S:\U_Proteomica\UNIDAD\software\MacrosRafa\data\Metabolomics\PESA_Integromics\Data\Metabolomics\ALDH4\OriginalFiles\RBR_Liver_Antibodies.xlsx"
f2i_path = r"S:\U_Proteomica\UNIDAD\software\MacrosRafa\data\Metabolomics\PESA_Integromics\Data\Metabolomics\ALDH4\OriginalFiles\f2i.xlsx"
mdata_path = r"S:\U_Proteomica\UNIDAD\software\MacrosRafa\data\Metabolomics\PESA_Integromics\Data\Metadata\ALDH4\WorkingFiles\main_metadata.tsv"
w_path = r"S:\U_Proteomica\UNIDAD\software\MacrosRafa\data\Metabolomics\PESA_Integromics\Data\Metabolomics\ALDH4\WorkingFiles"

In [3]:
#
# Read metadata
#

mdata = pd.read_csv(mdata_path, sep='\t')

In [4]:
#
# Generate f2i.tsv
#

f2i = DotMap({
    'cp': pd.read_excel(f2i_path, sheet_name='C18P'),
    'cn': pd.read_excel(f2i_path, sheet_name='C18N')
})

f2i = pd.concat([f2i.cp, f2i.cn])
f2i.to_csv(os.path.join(w_path, 'f2i.tsv'), sep='\t', index=False)

In [5]:
#
# 
#

xm = DotMap({
    'cp': pd.read_excel(xm_path, sheet_name='C18P').set_index('fid').T,
    'cn': pd.read_excel(xm_path, sheet_name='C18N').set_index('fid').T
})

qc = DotMap({
    'cp': pd.read_excel(xm_path, sheet_name='C18P_QC').set_index('fid').T,
    'cn': pd.read_excel(xm_path, sheet_name='C18N_QC').set_index('fid').T
})

In [6]:
#
# Remove from xm features with more than 1 missing value in any group
# In QC, remove if it has more than 1 missing value

thr = 1
g2sn = mdata.groupby('Group').agg(list).Seqn

xmf = DotMap()
qcf = DotMap()

for i in modes:
    print('Samples:')
    print(f'Features prior to filtration in {i}: {xm[i].shape[1]}')
    xmf[i] = xm[i].loc[
        :,
        ~np.logical_or.reduce(
            [xm[i].loc[j].isna().sum()>thr for j in g2sn]
        )
    ]
    print(f'Features after to filtration in {i}: {xmf[i].shape[1]}')
    print('QC:')
    print(f'Features prior to filtration in {i}: {qc[i].shape[1]}')
    qcf[i] = qc[i].loc[:, ~(qc[i].isna().sum()>1)]
    print(f'Features after filtration in {i}: {qcf[i].shape[1]}')
    print()

Samples:
Features prior to filtration in cp: 3409
Features after to filtration in cp: 2226
QC:
Features prior to filtration in cp: 3409
Features after filtration in cp: 2884

Samples:
Features prior to filtration in cn: 1185
Features after to filtration in cn: 709
QC:
Features prior to filtration in cn: 1185
Features after filtration in cn: 1096



In [7]:
#
# Apply MSTUS using features with no missing values
# Apply MSTUS to QC using the same features

#i = modes[0]

xmfr = DotMap()
qcr = DotMap()
for i in modes:
    mstus = xmf[i].loc[:, ~xmf[i].isna().any()].sum(axis=1)
    #mstus = xmf[i].sum(axis=1)
    xmfr[i] = pd.DataFrame(index=xmf[i].index, columns=xmf[i].columns)
    for j in range(xmfr[i].shape[1]):
        col = xmfr[i].columns[j]
        xmfr[i][col] = xmf[i].loc[:, col]/mstus

    # apply ratio to quality control using the same features used with samples
    mstusqc = qc[i].loc[:, xmf[i].columns[~xmf[i].isna().any()]].sum(axis=1)
    qcr[i]  = pd.DataFrame(index=qc[i].index, columns=qc[i].columns)
    for j in range(qcr[i].shape[1]):
        col = qcr[i].columns[j]
        qcr[i][col] = qc[i].loc[:, col]/mstusqc

In [8]:
#
# Check MSTUS in QC
#

In [9]:
files = ['Plots/MSTUS_Samples.html', 'Plots/MSTUS_QC.html']
xmdf = [xmf, xmfr]
qcdf = [qc, qcr]

for file, dflist in [(files[0], xmdf), (files[1], qcdf)]:

    if os.path.exists(file):
        os.remove(file)


    titles = ['C18P | Log', 'C18P | Log + MSTUS', 'C18N | Log', 'C18N | Log + MSTUS']

    fig0 = make_subplots(rows=2, cols=2, subplot_titles=titles, vertical_spacing=0.1) 
    fig1 = make_subplots(rows=2, cols=2, subplot_titles=titles, vertical_spacing=0.1)
    fig2 = make_subplots(rows=2, cols=2, subplot_titles=titles, vertical_spacing=0.1)


    for nr,i in enumerate(modes):
        for nc,df in enumerate(dflist):
            colors = n_colors('rgb(5, 200, 200)', 'rgb(200, 10, 10)', df[i].shape[0], colortype='rgb')
            for j,c in zip(df[i].index, colors):

                fig0.add_trace(go.Box(
                    #x=[j], 
                    y=np.log2(df[i].loc[j].to_numpy()),
                    name=j, 
                    showlegend=False, marker_color=c
                ), row=nr+1, col=nc+1)

                fig1.add_trace(go.Violin(
                    #x=[j], 
                    y=np.log2(df[i].loc[j].to_numpy()),
                    name=j, box_visible=True, meanline_visible=False,
                    line_color=c, showlegend=False, side=None
                ), row=nr+1, col=nc+1)

                fig2.add_trace(go.Violin(
                    x=np.log2(df[i].loc[j].to_numpy()),
                    side='positive', line_color=c, points=False, width=4, name=j, showlegend=False
                ),row=nr+1, col=nc+1)

    fig0.update_layout(height=1200, title='Boxplot')
    fig1.update_layout(height=1200, title='Violin plot')
    fig2.update_layout(height=1200, title='Density plot')
    #fig.show(); fig2.show()
    with open(file, 'a') as f:
                f.write(fig0.to_html(full_html=False, include_plotlyjs='cdn', default_height='50%', default_width='80%'))
                f.write(fig1.to_html(full_html=False, include_plotlyjs='cdn', default_height='50%', default_width='80%'))
                f.write(fig2.to_html(full_html=False, include_plotlyjs='cdn', default_height='50%', default_width='80%'))

In [10]:
#
# Apply log transformation
#

for i in modes:
    xmfr[i] = np.log2(xmfr[i])

In [11]:
from PlotEDA import PlotEDA
file = 'Plots/GroupsEffect_C18P.html'
if os.path.exists(file):
    os.remove(file)

plotEDA = PlotEDA(xmfr.cp, mdata, file=file)

plotEDA.plotSummary(r11=(-20,0), r12=(0,2), r21=(-0.4,0.4), r22=(0,2), r3=(-4,4),vl3=[0], binsize=0.1)
plotEDA.plotByGroup('Group', vl1=[0],vl2=[0], r1=(-20, 0),binsize=0.1)
plotEDA.plotByGroup('Ig', vl1=[0],vl2=[0], r1=(-20, 0),binsize=0.1)
plotEDA.plotByGroup('Control', vl1=[0],vl2=[0], r1=(-20, 0),binsize=0.1)

from PlotEDA import PlotEDA
file = 'Plots/GroupsEffect_C18N.html'
if os.path.exists(file):
    os.remove(file)

plotEDA = PlotEDA(xmfr.cn, mdata, file=file)

plotEDA.plotSummary(r11=(-20,0), r12=(0,2), r21=(-0.4,0.4), r22=(0,2), r3=(-4,4),vl3=[0], binsize=0.1)
plotEDA.plotByGroup('Group', vl1=[0],vl2=[0], r1=(-20, 0),binsize=0.1)
plotEDA.plotByGroup('Ig', vl1=[0],vl2=[0], r1=(-20, 0),binsize=0.1)
plotEDA.plotByGroup('Control', vl1=[0],vl2=[0], r1=(-20, 0),binsize=0.1)

In [12]:
#
# Combine cp and cn
#

xmfr = xmfr.cp.join(xmfr.cn)

In [13]:
#
# Center and scale
#

from sklearn.preprocessing import StandardScaler

xmfrn = pd.DataFrame(
    StandardScaler().fit_transform(xmfr),
    columns=xmfr.columns, index=xmfr.index
)

In [14]:
#
# Imputation of missing values using KNN
#

from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=3)
xmfrnv = pd.DataFrame(
    imputer.fit_transform(xmfrn),
    columns=xmfrn.columns,
    index=xmfrn.index
)

In [15]:
print(f"Total number of values: {xmfrnv.shape[0]*xmfrnv.shape[1]}")
print(f"Number of missing values imputed by KNN (n=3): {xmfrn.isna().sum().sum()} ({round(xmfrn.isna().sum().sum()/(xmfrnv.shape[0]*xmfrnv.shape[1])*100,2)}%)")

Total number of values: 52830
Number of missing values imputed by KNN (n=3): 1184 (2.24%)


In [20]:
#
# Write normalized 
#

xmfrnv.to_csv('Xm_norm.tsv', sep='\t')

In [17]:
from PCA_UMAP import PCA_UMAP, PCA_Var
file = 'Plots/PCA_UMAP.html'
if os.path.exists(file):
    os.remove(file)


pcaumap = PCA_UMAP(xmfrnv, mdata, file=file)
pcaumap.plotReduction('Group', pcacomp=[0,1])
pcaumap.plotReduction('Ig', pcacomp=[0,1])
pcaumap.plotReduction('Control', pcacomp=[0,1])

In [18]:
PCA_Var(xmfrnv, mdata, [], ['Group', 'Ig', 'Control'], n_comp=10)

,%Var PCA,Group,Ig,Control
1,22.050658,0.8587,0.7102,0.8600
2,16.459395,0.0367,0.7280,0.0167
3,12.338912,0.3340,0.4906,0.4384
4,8.429696,0.3099,0.7365,0.2635
5,5.649381,0.0002,0.0000,0.1448
6,5.346861,0.3513,0.6351,0.1487
7,4.351831,0.5658,0.7179,0.2858
8,3.591798,0.6427,0.9580,0.4258
9,3.295505,0.8966,0.6580,0.9425
10,2.937202,0.5763,0.5623,0.2866
